### Importing CSV file into postgres datbase

#### Steps

- import the CSV file into a pandas df
- clean the table  and remove all extra symplos, spaces , capital lettters
- write and creaate SQL statement 
- import the data into the database 


In [2]:
# import labraries
import os
import numpy as np
import pandas as pd
import psycopg2


In [3]:
!ls

Customer Contracts$.csv
Postgre2python.py
Untitled.ipynb
spy2post.py
sql


In [5]:
df = pd.read_csv('Customer Contracts$.csv')
df.head(5)


,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [8]:
# clean table names 
# lower cases letters
# remove all white spaces 
file = 'Customer Contracts$'

clean_tbl_name = file.lower().replace(" ","_").replace("?","").replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("#", "").replace("|", "").replace(r")", "").replace(r"(", "").replace("$","")

clean_tbl_name

'customer_contracts'

In [9]:
df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [11]:
df.columns = [x.lower().replace(" ","_").replace("?","").replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("#", "").replace("|", "").replace(r")", "").replace(r"(", "").replace("$","") for x in df.columns]

In [12]:
df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [ ]:
create table customer_contracts
(
    customer_name varchar
    start_date    varchar
    end_date      varchar
    contract_amount_m  float
    invoice_sent  varchar
    paid          varchar 
);

In [13]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [14]:
replacement = {
    "object": "varchar",
    "float64": "float",
    "int64": "int",
    "datetime64": "timestamp",
    "timedelta64[ns]": "varchar"
}

In [15]:
replacement

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [19]:
col_str = ", ".join("{} {}".format(a, d) for (a, d) in zip(df.columns, df.dtypes.replace(replacement)))

In [20]:
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [21]:
#db settings
#host = 'add'
#dbname = 'add db name'
#user = 'add db username'
#password = 'add db pwd'
hostname = 'localhost'
database = 'demo'
username = 'postgres'
pwd = ''



In [22]:
conn = psycopg2.connect(host= hostname,
            port = port_id,
            dbname = database,
            user = username,
            password = pwd) 

In [26]:
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS customer_contracts')

### Create a table in postgreSQL

In [27]:

create_script = ''' CREATE TABLE customer_contracts (
                        customer_name varchar, 
                        start_date varchar, 
                        end_date varchar, 
                        contract_amount_m float, 
                        invoice_sent varchar, 
                        paid varchar
                        ) '''

cur.execute(create_script)

In [29]:
# save df  to csv file
df.to_csv('customer_contracts.csv', header = df.columns, index=False, encoding="utf-8")

In [30]:
#open the csv file save if as objects and uploads to db
my_file = open("customer_contracts.csv")
print("file opened!")

file opened!


### Notes
- Use COMMIT to successfully terminate a transaction.

In [40]:
conn.commit()

SQL_STATEMENT = """
COPY customer_contracts FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""
cur.copy_expert(sql=SQL_STATEMENT, file=my_file)
